In [96]:
#Import beautifulsop and requests
from bs4 import BeautifulSoup
import requests

In [97]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [98]:
soup=BeautifulSoup(source,'lxml')

In [99]:
#Check title page
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [100]:
#Apply Prettify() function in BeautifulSoup to enable us to view how the tags are nested in the document.
#print(soup.prettify())

In [101]:
#Scrap the table:
table=soup.find('table',{"class":'wikitable'})
#table

In [102]:
table_rows = table.find_all('tr')

In [103]:
#Get the dataframe
import pandas as pd
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [104]:
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [105]:
df.shape

(181, 3)

## Preprocessing data. To gnore cells with a borough that is Not assigned.

In [106]:
#Drop first none row
df.drop([0],inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


In [107]:
df.drop(df[df['Borough']=='Not assigned'].index,axis=0,inplace=True)
df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Malvern / Rouge
12,M3B,North York,Don Mills
13,M4B,East York,Parkview Hill / Woodbine Gardens
14,M5B,Downtown Toronto,"Garden District, Ryerson"


In [108]:
df.shape

(103, 3)

## Q2, to utilize location data

In [109]:
#!conda install -c conda-forge geocoder --yes

In [110]:
#conda update -n base -c defaults conda

In [111]:
#Use the Geocoder package or the csv file to create dataframe with longitude and latitude values
url='http://cocl.us/Geospatial_data'
df_1=pd.read_csv(url)
df_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [112]:
#Be careful that the label 'Postal Code' does not match with what we wnat to merge, so that you need to rename your column name
df_2=df_1.rename(columns={"Postal Code": "PostalCode"},)
df_2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [113]:
df_part2=pd.merge(df,df_2[['PostalCode','Latitude', 'Longitude']],on='PostalCode')
df_part2.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
